## 1. Download the customer data set and add it to your “Original Data” folder.
## 2. Create a new notebook in your “Scripts” folder for part 1 of this task.

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
path = r'/Users/carolinazea/OneDrive/CareerFoundry/Achievement 4- Python/Instacart Basket Analysis'

## 3. Import your analysis libraries, as well as your new customer data set as a dataframe

In [4]:
cust_data_set = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

## 4. Wrangle the data so that it follows consistent logic; for example, rename columns with illogical names and drop columns that don’t add anything to your analysis.
## 5. Complete the fundamental data quality and consistency checks you’ve learned throughout this Achievement; for example, check for and address missing values and duplicates, and convert any mixed-type data

## (Checking data quality check for imported 'customer data set')

In [5]:
# view first five columns of df

cust_data_set.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
# check df for column and row size

cust_data_set.shape

(206209, 10)

In [6]:
# check for descriptive statistics 

cust_data_set.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [6]:
# rename 'surnam' to 'last name' as stakeholders might wants to see 'last name' followed by 'first name'

cust_data_set.rename(columns = {'Surnam' : 'Last Name'}, inplace = True)

In [7]:
# check for missing values 

cust_data_set.isnull().sum()

user_id             0
First Name      11259
Last Name           0
Gender              0
STATE               0
Age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

In [8]:
# remove missing values as they make up less than 5% of data

customer_data = cust_data_set[cust_data_set['First Name'].isnull() == False]

In [9]:
# recheck columns and row size 

customer_data.shape

(194950, 10)

In [11]:
# check for duplicates

cust_data_set_dups = customer_data[customer_data.duplicated()]

In [12]:
cust_data_set_dups

,user_id,First Name,Last Name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income


In [13]:
# check for mixed data types 

for col in customer_data.columns.tolist():
    weird = (customer_data[[col]].applymap(type) != customer_data[[col]].iloc[0].apply(type)).any(axis = 1)
    if len(customer_data[weird]) > 0:
        print(col)

## 6. Combine your customer data with the rest of your prepared Instacart data. Tip: Make sure the key columns are of the same data type

In [10]:
# import ords_prods_merge dataframe 

ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_groupingdata.pkl'))

In [17]:
# check order/product df shape

ords_prods_merge.shape

(32404859, 23)

In [11]:
#check columns of order/product df

ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_range_loc', 'busiest_day',
       'Busiest_slowest_days', 'busiest_period_of_day', 'max_order',
       'loyalty_flag', 'avg_price', 'spending_flag', 'median_order',
       'order_frequency_flag'],
      dtype='object')

In [12]:
#check columns of customer df
customer_data.columns

Index(['user_id', 'First Name', 'Last Name', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [13]:
# customer_prods will be the df that merges 'ords_prods_merge' & 'customer_data'
customerords_prods = ords_prods_merge.merge(customer_data, on = 'user_id')

In [20]:
#recheck columns to see if merge was successful 
customerords_prods.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_range_loc', 'busiest_day',
       'Busiest_slowest_days', 'busiest_period_of_day', 'max_order',
       'loyalty_flag', 'avg_price', 'spending_flag', 'median_order',
       'order_frequency_flag', 'First Name', 'Last Name', 'Gender', 'STATE',
       'Age', 'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [21]:
# recheck first 50 rows of newly merged dataframe

customerords_prods.head(50)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_slowest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_order,order_frequency_flag,First Name,Last Name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,11.114836,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.000000,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.000000,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.000000,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest Days,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.000000,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest Days,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
5,3367565,1,6,2,7,19.000000,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
6,550135,1,7,1,9,20.000000,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
7,3108588,1,8,1,14,14.000000,196,2,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
8,2295261,1,9,1,16,0.000000,196,4,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
9,2550362,1,10,4,8,30.000000,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest Days,Average orders,10,New customer,6.367797,Low spender,20.0,Regular customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [23]:
# recheck last 50 rows of dataframe

customerords_prods.tail(50)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_slowest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,median_order,order_frequency_flag,First Name,Last Name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
30629691,774135,11534,12,1,13,7.000000,42387,1,1,Natural 8 fl oz Alpine Spring Water,115,7,10.5,Mid-range product,Regularly busy,Regularly busy,Average orders,12,Regular customer,10.500000,High spender,7.000000,Frequent customer,Ann,Suarez,Female,Delaware,69,2/20/2018,0,divorced/widowed,103023
30629692,1280763,105857,1,1,22,11.114836,23767,1,0,Imported Light Beer,27,5,12.0,Mid-range product,Regularly busy,Regularly busy,Average orders,6,New customer,12.000000,High spender,9.500000,Frequent customer,Rose,Whitehead,Female,Oregon,28,8/28/2019,2,married,48873
30629693,593283,105857,2,6,21,12.000000,23767,1,1,Imported Light Beer,27,5,12.0,Mid-range product,Regularly busy,Regularly busy,Average orders,6,New customer,12.000000,High spender,9.500000,Frequent customer,Rose,Whitehead,Female,Oregon,28,8/28/2019,2,married,48873
30629694,2645685,105857,3,2,22,10.000000,23767,1,1,Imported Light Beer,27,5,12.0,Mid-range product,Regularly busy,Regularly busy,Average orders,6,New customer,12.000000,High spender,9.500000,Frequent customer,Rose,Whitehead,Female,Oregon,28,8/28/2019,2,married,48873
30629695,2934579,105857,4,3,2,8.000000,23767,1,1,Imported Light Beer,27,5,12.0,Mid-range product,Regularly busy,Regularly busy,Average orders,6,New customer,12.000000,High spender,9.500000,Frequent customer,Rose,Whitehead,Female,Oregon,28,8/28/2019,2,married,48873
30629696,1654464,105857,5,3,22,7.000000,23767,1,1,Imported Light Beer,27,5,12.0,Mid-range product,Regularly busy,Regularly busy,Average orders,6,New customer,12.000000,High spender,9.500000,Frequent customer,Rose,Whitehead,Female,Oregon,28,8/28/2019,2,married,48873
30629697,2494306,105857,6,5,21,9.000000,23767,1,1,Imported Light Beer,27,5,12.0,Mid-range product,Regularly busy,Regularly busy,Average orders,6,New customer,12.000000,High spender,9.500000,Frequent customer,Rose,Whitehead,Female,Oregon,28,8/28/2019,2,married,48873
30629698,3320476,38979,1,1,11,11.114836,13128,1,0,Purified Alkalkine Water with Minerals pH10,115,7,8.9,Mid-range product,Regularly busy,Regularly busy,Most orders,3,New customer,8.900000,Low spender,7.000000,Frequent customer,Annie,Goodwin,Female,Indiana,57,12/15/2019,1,married,135623
30629699,2054093,38979,2,1,10,7.000000,13128,1,1,Purified Alkalkine Water with Minerals pH10,115,7,8.9,Mid-range product,Regularly busy,Regularly busy,Average orders,3,New customer,8.900000,Low spender,7.000000,Frequent customer,Annie,Goodwin,Female,Indiana,57,12/15/2019,1,married,135623
30629700,3292079,38979,3,1,11,7.000000,13128,1,1,Purified Alkalkine Water with Minerals pH10,115,7,8.9,Mid-range product,Regularly busy,Regularly busy,Most orders,3,New customer,8.900000,Low spender,7.000000,Frequent customer,Annie,Goodwin,Female,Indiana,57,12/15/2019,1,married,135623


In [22]:
# save file as 'customer_ords_prods' in pkl file 

customerords_prods.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'customer_ords_prods.pkl'))

### I tried numerous times to include '.indicator = True' when merging the two data sets so i can view the valuecounts after merging, as well use 'how = 'right' but my notebook kept crashing and a dead kernel reapperared.

### So for this notebook, I left out these functions and just used the 'user_id' to merge these two datasets 